In [1]:
# librerías
import os
import pickle
import pandas as pd
import gower
from sklearn.svm import SVC
from sklearn.model_selection import  GridSearchCV, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
from sklearn.manifold import MDS
import gc

In [14]:
# directorios
dataset_dir = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'datasets')
image_path = os.path.join(os.path.dirname(os.path.dirname(os.path.abspath(''))), 'images')

In [26]:
# dataset b (sin llamante_edad y con casos completos de victima_edad)

llamados_v2= pd.read_excel(os.path.join(dataset_dir, 'xlsx/llamados_v2.xlsx'))

In [27]:
# quitar llamante edad
llamados_v2.drop(['llamante_edad'], axis=1, inplace=True) 

In [28]:
dataset_b = llamados_v2[~llamados_v2['victima_edad'].isnull()]

In [29]:

dataset_b.loc[:, 'victima_convive_agresor'] = dataset_b['victima_convive_agresor'].replace({'NS/NC': pd.NA})


In [30]:
# mapear SI NO a 1 0
dataset_b.loc[:,'victima_convive_agresor'] = dataset_b.loc[:,'victima_convive_agresor'].map({'SI': 1, 'NO': 0})


In [32]:
# chequear proporciones previas
print('prop. SI:', len(dataset_b[dataset_b['victima_convive_agresor']==1])/len(dataset_b)*100)
print('prop. NO:',len(dataset_b[dataset_b['victima_convive_agresor']==0])/len(dataset_b)*100)

prop. SI: 15.15958987429763
prop. NO: 67.59543532410358


In [33]:
dataset_b.loc[:,'victima_convive_agresor'].isna().sum()

2977

In [34]:
# separar features de target
X = dataset_b.drop(['victima_convive_agresor'], axis=1)
y_previo = dataset_b['victima_convive_agresor']

In [35]:
# guardar los índices de casos NSNC (vacíos) y sacaros del target (son solo para el test final)
nsnc_indices = y_previo[y_previo.isna()].index
y = y_previo.drop(nsnc_indices)

In [ ]:
# gower de X
gower_X = gower.gower_matrix(X)
print("gower para dataset_b hecho")

lista_dimensiones=[2,3,4,5,6,7,20]
lista_modelos=[]
contador_exp= 1
for i in lista_dimensiones:
    print('n_components: ', i)
    # correr NMDS sobre el total del dataset
    nmds = MDS(n_components=i ,metric=False, dissimilarity='precomputed', max_iter=300, random_state=0, normalized_stress=True) 
    X_nmds = nmds.fit_transform(gower_X)
    print('NMDS fit tr. hecho')

    # crear el test final con lo que corresponde a target de X transformado 
    test_final = X_nmds[nsnc_indices, :]
    test_final_df = pd.DataFrame(test_final, columns=[f'Component_{j+1}' for j in range(test_final.shape[1])])


    print('Stress: ' + str(round(nmds.stress_,2)))
    # quitar índices (el test final) para entrenar 
    X_nmds_limpio = pd.DataFrame(X_nmds).drop(nsnc_indices).reset_index(drop=True)
    y_limpio = y.reset_index(drop=True)

    print('entrenando el modelo svm')
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=2)
    for train_index, test_index in sss.split(X_nmds_limpio, y_limpio):
        X_train, X_test = X_nmds_limpio.iloc[train_index], X_nmds_limpio.iloc[test_index]
        y_train, y_test = y_limpio.iloc[train_index], y_limpio.iloc[test_index]

    # Parámetros de gridsearch
    param_grid = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto']
    }


    svm = SVC(class_weight='balanced')


    # GridSearchCV
    grid_search = GridSearchCV(svm, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    # Mejor modelo
    best_model = grid_search.best_estimator_

    # Aplicar al test set
    y_pred = best_model.predict(X_test)

    # Evaluate the model
    #print("Best parameters found: ", grid_search.best_params_)
    #print("Best cross-validation accuracy: ", grid_search.best_score_)
    #print("Test set accuracy: ", accuracy_score(y_test, y_pred))
    #print("Classification report:\n", classification_report(y_test, y_pred))

    # por cada modelo me guardo: n_components usado, best parameters, best cross validation acc, test set acc, classification report

    contador_exp=+1
    
    lista_modelos.append( {'Experimento':contador_exp,'N Components': i, "Mejores parámetros":grid_search.best_params_, 
                           "Mejor acc. en cross validation": grid_search.best_score_, 
                           'Test set acc.': accuracy_score(y_test, y_pred), "Classification report": classification_report(y_test, y_pred)
                           })

    gc.collect()


In [ ]:
# Testing the best model
na_predictions = best_model.predict(test_final)
test_final_df['victima_convive_agresor_pred'] = na_predictions